# Import

In [1]:
# Клонируем репозитерий stylegan2-ada-pytorch
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git /content/stylegan2-ada-pytorch

Cloning into '/content/stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 131 (delta 0), reused 0 (delta 0), pack-reused 129 (from 2)
Receiving objects: 100% (131/131), 1.13 MiB | 37.35 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
# Скачиваем модель StyleGAN2
!mkdir -p /content/StyleGAN2
!wget -q https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl -O /content/StyleGAN2/ffhq.pkl # модель обученная на датасете ffhq(лица)

In [ ]:
!pip install -q git+https://github.com/openai/CLIP.git # установка CLIP
!pip install -q ninja # утилита для сборки C++/CUDA кода
!pip install -q youtokentome # библиотека от Яндекса для работы с BPE-токенизатором

In [ ]:
import sys
import os
sys.path.append('/content/stylegan2-ada-pytorch')
os.environ['TORCH_CUDA_ARCH_LIST'] = '7.5'

import pickle
import copy

import numpy as np

import clip

import torch
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast

from torchvision.utils import make_grid, save_image
from torchvision.transforms import transforms

from PIL import Image
import matplotlib.pyplot as plt

import re

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

RAND = 2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Загружаем свое фото
#image =Image.open('/content/kot.jpg')

#plt.figure(figsize=(8,8))
#plt.imshow(image)
#plt.axis('off')
#plt.show()

# Utils

In [ ]:
TEMPLATES = [
    'a bad photo of a {}.',
    'a sculpture of a {}.',
    'a photo of the hard to see {}.',
    'a low resolution photo of the {}.',
    'a rendering of a {}.',
    'graffiti of a {}.',
    'a bad photo of the {}.',
    'a cropped photo of the {}.',
    'a tattoo of a {}.',
    'the embroidered {}.',
    'a photo of a hard to see {}.',
    'a bright photo of a {}.',
    'a photo of a clean {}.',
    'a photo of a dirty {}.',
    'a dark photo of the {}.',
    'a drawing of a {}.',
    'a photo of my {}.',
    'the plastic {}.',
    'a photo of the cool {}.',
    'a close-up photo of a {}.',
    'a black and white photo of the {}.',
    'a painting of the {}.',
    'a painting of a {}.',
    'a pixelated photo of the {}.',
    'a sculpture of the {}.',
    'a bright photo of the {}.',
    'a cropped photo of a {}.',
    'a plastic {}.',
    'a photo of the dirty {}.',
    'a jpeg corrupted photo of a {}.',
    'a blurry photo of the {}.',
    'a photo of the {}.',
    'a good photo of the {}.',
    'a rendering of the {}.',
    'a {} in a video game.',
    'a photo of one {}.',
    'a doodle of a {}.',
    'a close-up photo of the {}.',
    'a photo of a {}.',
    'the origami {}.',
    'the {} in a video game.',
    'a sketch of a {}.',
    'a doodle of the {}.',
    'a origami {}.',
    'a low resolution photo of a {}.',
    'the toy {}.',
    'a rendition of the {}.',
    'a photo of the clean {}.',
    'a photo of a large {}.',
    'a rendition of a {}.',
    'a photo of a nice {}.',
    'a photo of a weird {}.',
    'a blurry photo of a {}.',
    'a cartoon {}.',
    'art of a {}.',
    'a sketch of the {}.',
    'a embroidered {}.',
    'a pixelated photo of a {}.',
    'itap of the {}.',
    'a jpeg corrupted photo of the {}.',
    'a good photo of a {}.',
    'a plushie {}.',
    'a photo of the nice {}.',
    'a photo of the small {}.',
    'a photo of the weird {}.',
    'the cartoon {}.',
    'art of the {}.',
    'a drawing of the {}.',
    'a photo of the large {}.',
    'a black and white photo of a {}.',
    'the plushie {}.',
    'a dark photo of a {}.',
    'itap of a {}.',
    'graffiti of the {}.',
    'a toy {}.',
    'itap of my {}.',
    'a photo of a cool {}.',
    'a photo of a small {}.',
    'a tattoo of the {}.'
]

In [ ]:
def cleaned_text(text) -> str:
    """
    Простая очистка текста.
    """
    text = str(text) if text is not None else '' # преобразование text в строку, если это не строка
    text = text.lower()
    text = re.sub(r'[^а-яёa-z0-9\s.,*!?:-]', '', text)  # удаление лишних символов (кроме пунктуации)
    text = re.sub(r'\s+', ' ', text).strip()  # удаление лишних пробелов
    text = re.sub(r'^[^\w]+', '', text)  # удаление пунктуации в начале строки
    text = text.strip(' .')  # убирает лишние пробелы и точки в начале и в конце строки
    return text

In [ ]:
def make_prompt_list(word: str):
  """
  Создает список промптов.
  """
  word = cleaned_text(word)
  return [t.format(word) for t in TEMPLATES]

In [ ]:
def load_clip_models(model_names: list[str],
                     device: torch.device,
                     weights: list[float]=[1.0, 1.0]) -> list:
  """
  Загружает модели и препроцессоры CLIP.

  :param model_names: список имен моделей CLIP
  :param device: устройство для вычислений (CPU или CUDA)
  :param weights: вклад модели CLIP в итоговый эмбеддинг

  :return: список моделей, препроцессоров и весов
  """
  models = []

  for name, w in zip(model_names, weights):
    model, preprocess = clip.load(name, device=device)
    for p in model.parameters():
      p.requires_grad = False
    models.append((model, preprocess, w))

  return models

In [ ]:
def preprocessing_text(models: list,
                       prompts: list[str],
                       device: torch.device) -> torch.Tensor:
  """
  Принимает список текстовых промптов,
  возвращает один усреднённый и нормализованный эмбеддинг.

  :param models: список моделей, препроцессоров и весов CLIP
  :param prompts: список текстовых промптов
  :param device: устройство для вычислений (CPU или CUDA)

  :return: эмбеддинг текста в пространстве CLIP
  """
  total_embedding = 0.0
  weights_sum = 0.0

  for model in models:
    weights_sum += model[2]

  tokens = clip.tokenize(prompts).to(device)

  for model in models:
    model[0].eval()
    text_features = model[0].encode_text(tokens)
    # Нормализация
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    # Усреднение всех промптов
    text_mean = text_features.mean(dim=0, keepdim=True)
    # Нормализация
    text_mean = text_mean / text_mean.norm(dim=-1, keepdim=True)
    # Эмбеддинг с учетом веcа модели
    total_embedding += text_mean * (model[2] / weights_sum)

  # Нормализация
  total_embedding = total_embedding / total_embedding.norm(dim=-1, keepdim=True)

  return total_embedding.detach()

In [ ]:
def preprocessing_image(models: list,
                        image,
                        device: torch.device) -> torch.Tensor:
  """
  Если image - torch.Tensor, то преобразует его в PIL.
  Если image — PIL.Image, используется стандартный CLIP-препроцессор.
  Возвращает нормализованный эмбеддинг изображения.

  :param models: список моделей, препроцессоров и весов CLIP
  :param image: входное изображение
  :param device: устройство для вычислений (CPU или CUDA)

  :return: эмбеддинг изображения в пространстве CLIP
  """

  total_embedding = 0.0
  weights_sum = 0.0

  for model in models:
    weights_sum += model[2]

  if isinstance(image, torch.Tensor):
    # Для torch.Tensor из StyleGAN2 с диапазоном [-1,1]
    transform = transforms.Compose([
        transforms.Lambda(lambda x: ((x + 1) / 2).clamp(0, 1)),
        transforms.Lambda(lambda x: F.interpolate(x,
                                                  size=(224, 224),
                                                  mode='bicubic',
                                                  align_corners=False)),
        transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073),
                             std=(0.26862954, 0.26130258, 0.27577711))])

    image_preproc = transform(image.to(device))

    for model in models:
      model[0].eval()
      image_features = model[0].encode_image(image_preproc)
      # Нормализация
      image_features = image_features / image_features.norm(dim=-1, keepdim=True)
      # Эмбеддинг с учетом веcа модели
      total_embedding += image_features * (model[2] / weights_sum)

    # Нормализация
    total_embedding = total_embedding / total_embedding.norm(dim=-1, keepdim=True)

  else:
    # Для PIL-изображений
    for model in models:
      model.eval()
      image_preproc = model[1](image).unsqueeze(0).to(device)
      image_features = model[0].encode_image(image_preproc)

      # Нормализация
      image_features = image_features / image_features.norm(dim=-1, keepdim=True)
      # Эмбеддинг с учетом веcа модели
      total_embedding += image_features * (model[2] / weights_sum)

    # Нормализация
    total_embedding = total_embedding / total_embedding.norm(dim=-1, keepdim=True)

  return total_embedding

In [ ]:
def extract_patches(image: torch.Tensor,
                    patch_size: int=64,
                    num_patches: int=8):
    """
    Извлекает случайные патчи из изображения.

    :param image: тензор [B, C, H, W]
    :param patch_size: размер квадрата патча
    :param num_patches: количество патчей

    :return: тензор [B*num_patches, C, patch_size, patch_size]
    """
    B, C, H, W = image.shape
    patches = []
    for _ in range(num_patches):
        i = torch.randint(0, H - patch_size, (1,)).item()
        j = torch.randint(0, W - patch_size, (1,)).item()
        patch = image[:, :, i:i+patch_size, j:j+patch_size]

        patches.append(patch)

    return torch.cat(patches, dim=0)

In [ ]:
def make_latents(G,
                 RAND: int,
                 device: torch.device):
  """
  Герерирует латенты.

  :param G: генератор, используется для определение размера вектора
  :param RAND: фиксирует генератор случайных чисел
  :param device: устройство для вычислений (CPU или CUDA)

  :return: два латента (latent, latent_sample)
  """
  torch.cuda.manual_seed(RAND)
  latent_sample = torch.randn(4, G.z_dim, device=device)
  latent = torch.randn(2, G.z_dim, device=device)
  latent_plus = torch.randn(8, G.z_dim, device=device)

  return latent_sample, latent, latent_plus

# StyleGN2-ADA

In [ ]:
with open('/content/StyleGAN2/ffhq.pkl', 'rb') as f:
  G = pickle.load(f)['G_ema'].to(device)

G_frozen = copy.deepcopy(G) # копия генератора (замороженный)

In [11]:
latent_sample, latent, latent_plus = make_latents(G,
                                                  RAND=RAND,
                                                  device=device)

c = None
G.eval()
image_gan = G(latent_sample,
              c,
              truncation_psi=0.7,
              noise_mode='const')

Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.


In [ ]:
img = ((image_gan +  1) / 2).clamp(0, 1)  # [-1, 1] → [0, 1]
grid = make_grid(img, nrow=2)
plt.figure(figsize=(8, 8))
plt.imshow(grid.permute(1, 2, 0).detach().cpu().numpy())
plt.axis('off')
plt.show()

# CLIP

In [14]:
# Загружаем модель и процессор
model, preprocess = clip.load('ViT-B/32', device=device)

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 97.4MiB/s]


In [15]:
text_features = preprocessing_text(model, 'woman', device)

In [16]:
image_features = preprocessing_image(model, preprocess, image_gan, device)

# Train

## Loss

### cosine_dist

In [ ]:
def cosine_dist(image_features: torch.Tensor,
                text_features: torch.Tensor) -> torch.Tensor:
 """
 Вычисляет глобальный CLIP-loss (cosine distance).

 :param image_features: вектора изображения в пространстве CLIP
 :param text_features: вектора текста в пространстве CLIP

 :return: косинусное расстояние
 """
 global_loss = 1 - F.cosine_similarity(image_features, text_features, dim=-1)

 return global_loss.mean()

### clip_loss

In [ ]:
def delta_text(models: list,
               text_target: str,
               text_source: str,
               device: torch.device) -> torch.Tensor:
  """
  Вычисляет вектор текстового направления
  между таргетом и текущим стилем для множества промптов.

  :param models: список моделей, препроцессоров и весов CLIP
  :param text_target: текст целевого стиля
  :param text_source: текст исходного стиля
  :param device: устройство для вычислений (CPU или CUDA)

  :return: вектор текстового направления
  """

  prompts_src = make_prompt_list(text_source)
  prompts_tgt = make_prompt_list(text_target)

  txt_src = preprocessing_text(models, prompts_src, device)
  txt_tgt = preprocessing_text(models, prompts_tgt, device)

  delta = txt_tgt - txt_src
  delta = delta / delta.norm(dim=-1, keepdim=True)

  return delta

In [ ]:
def delta_image(models: list,
                image: torch.Tensor,
                image_frozen: torch.Tensor,
                device: torch.device) -> torch.Tensor:
  """
  Вычисляет вектор визуального направления
  между изображением до и после fine-tuning.

  :param models: список моделей, препроцессоров и весов CLIP
  :param image: изображение из обучаемого генератора
  :param image_frozen: изображение из замороженного генератора
  :param device: устройство для вычислений (CPU или CUDA)

  :return: вектор визуального направления
  """
  img = preprocessing_image(models, image, device)
  img_frz = preprocessing_image(models, image_frozen, device)

  delta = img - img_frz
  delta = delta / delta.norm(dim=-1, keepdim=True)

  return delta

In [ ]:
def clip_loss(delta_image: torch.Tensor,
              delta_text: torch.Tensor) -> torch.Tensor:
  """
  Вычисляет направленный CLIP-loss.

  :param delta_image: вектор визуального направления
  :param delta_text: вектор текстового направления

  :return: направленный CLIP-loss (скаляр)
  """
  direction_clip_loss = 1 - F.cosine_similarity(delta_image, delta_text, dim=-1)
  return direction_clip_loss.mean()

## Train

### clip_loss

In [ ]:
def train_generator(generator,
                    generator_frozen,
                    model_names: list[str],
                    text_source: str,
                    text_target: str,
                    epochs: int,
                    criterion,
                    device: torch.device,
                    RAND: int=2,
                    batch_size: int=2,
                    sample_size: int=4,
                    batch_size_w: int=8,
                    lr: float=0.002,
                    lr_w: float=0.02,
                    k: int=12,
                    lambda_patch: float=0.1,
                    lambda_l2: float=0.001,
                    max_norm: float=1.0,
                    weights: list[float]=[1.0, 1.0],
                    use_conv_layers: bool=False,
                    c: torch.Tensor=None):
  """
  Обучает генератор StyleGAN2 для изменения изображений в направлении текста
  с использованием CLIP-косинусного расстояния в качестве функции потерь.

  Функция выполняет fine-tuning генератора на фиксированном латентном векторе,
  вычисляет векторное направление между изображениями до и после обучения
  и текстовыми эмбеддингами, и минимизирует косинусное расстояние между ними.

  Каждые 50 эпох сохраняет сгенерированные изображения.
  В конце визуализирует график лосса.

  :param generator: генератор StyleGAN2, который будет обучаться
  :param generator_frozen: замороженный генератор, используемый для вычисления delta_image
  :param model_names: список имен моделей CLIP
  :param text_source: исходный текстовый промпт
  :param text_target: целевой текстовый промпт
  :param epochs: количество эпох обучения генератора
  :param criterion: функция потерь, которая принимает delta_image и delta_text и возвращает скалярный loss
  :param device: устройство для вычислений (CPU или CUDA)
  :param RAND: фиксирует генератор случайных чисел
  :param batch_size: размерность батча для обучения
  :param sample_size: размерность генерируемого сэмпла
  :param batch_size_w: размерность батча для оптимизации векторов из пространства W+
  :param lr: скорость обучения для оптимизатора параметров генератора
  :param lr_w: скорость обучения для оптимизатора векторов из пространства W+
  :param k: количество слоев которые обучаются
  :param lambda_patch: коэффициент патч-регуляризации
  :param lambda_l2: коэффициент l2-регуляризации
  :param max_norm: максимальная норма градиентов
  :param weights: вклад модели CLIP в итоговый эмбеддинг
  :param use_conv_layers: флаг для разморозки сверточных слоев генератора
  :param c: условие для генератора

  :return: None
  """
  loss_lst = []

  torch.cuda.empty_cache()
  torch.cuda.manual_seed(RAND)
  os.makedirs('results', exist_ok=True)

  latent_sample = torch.randn(sample_size, generator.z_dim, device=device)

  models = load_clip_models(model_names,
                            device=device,
                            weights=weights)

  generator = generator.to(device)
  generator_frozen = generator_frozen.to(device)
  generator.train()
  generator_frozen.eval()

  lst_block = []
  for name, module in generator.synthesis.named_children():
    if name.startswith('b'):
        lst_block.append(name)

  # Заморозка всего generator_frozen
  for param in generator_frozen.parameters():
    param.requires_grad = False

  # Заморозка mapping generator
  for param in generator.mapping.parameters():
    param.requires_grad = False

  optimizer = torch.optim.Adam(generator.synthesis.parameters(), lr=lr)

  for epoch in tqdm(range(epochs)):

    # Оптимизация векторов из пространства W+
    latent_plus = torch.randn(batch_size_w, generator.z_dim, device=device)
    w_plus_source = generator.mapping(latent_plus,
                                      c) # [batch_size, num_layers, dim]
    w_plus_target = generator.mapping(latent_plus,
                                      c) # [batch_size, num_layers, dim]
    w_plus_target = w_plus_target.clone().requires_grad_(True)

    optimizer_w_plus = torch.optim.Adam([w_plus_target], lr=lr_w)

    optimizer_w_plus.zero_grad()

    image_w_plus = generator.synthesis(w_plus_target,
                                       noise_mode='const')

    image_w_plus_features = preprocessing_image(models,
                                                image_w_plus,
                                                device)

    prompts_tgt = make_prompt_list(text_target)

    text_w_plus_features = preprocessing_text(models,
                                              prompts_tgt,
                                              device)

    loss_w_plus = cosine_dist(image_w_plus_features,
                              text_w_plus_features)

    loss_w_plus.backward()

    # Ограничивает норму градиентов
    torch.nn.utils.clip_grad_norm_([w_plus_target], max_norm)

    optimizer_w_plus.step()

    delta_w_plus = w_plus_target.detach() - w_plus_source.detach()
    delta_norm = delta_w_plus.mean(dim=0).norm(p=2, dim=-1)

    topk_indices = torch.topk(delta_norm, k).indices

    # Удаляет отработанные тензоры
    del latent_plus, w_plus_source, w_plus_target, image_w_plus
    del image_w_plus_features, text_w_plus_features, loss_w_plus, delta_w_plus, delta_norm

    torch.cuda.empty_cache()

    # Заморозка synthesis
    for p in generator.synthesis.parameters():
      p.requires_grad = False

    # Заморозка affine и torgb параметров
    #for name, module in generator.synthesis.named_modules():
    #  lname = name.lower()
    #  if 'affine' in lname or 'torgb' in lname:
    #    for p in module.parameters():
    #      p.requires_grad = False

    # Заморозка всех слоев кроме топ-k
    for ind in topk_indices:
      block_idx = ind // 2
      conv_num = ind % 2

      block_name = lst_block[block_idx]
      block = getattr(generator.synthesis, block_name)

      if use_conv_layers:
      # Заморозка всех слоев кроме топ-k conv слоев
        if block_name == 'b4':
          block.conv1.weight.requires_grad = True
          block.conv1.bias.requires_grad = True
        else:
          if conv_num == 0:
            block.conv0.weight.requires_grad = True
            block.conv0.bias.requires_grad = True
          else:
            block.conv1.weight.requires_grad = True
            block.conv1.bias.requires_grad = True
      else:
        # Заморозка всех слоев кроме топ-k affine и conv слоев
        if block_name == 'b4':
          block.conv1.weight.requires_grad = True
          block.conv1.bias.requires_grad = True
          block.conv1.affine.weight.requires_grad = True
          block.conv1.affine.bias.requires_grad = True
        else:
          if conv_num == 0:
            block.conv0.weight.requires_grad = True
            block.conv0.bias.requires_grad = True
            block.conv0.affine.weight.requires_grad = True
            block.conv0.affine.bias.requires_grad = True
          else:
            block.conv1.weight.requires_grad = True
            block.conv1.bias.requires_grad = True
            block.conv1.affine.weight.requires_grad = True
            block.conv1.affine.bias.requires_grad = True

    train_params = [param for param in generator.parameters() if param.requires_grad]
    # обновляет параметры оптимизатора
    optimizer.param_groups[0]['params'] = train_params
    #optimizer = torch.optim.Adam(train_params, lr=lr)

    optimizer.zero_grad()

    latent = torch.randn(batch_size, generator.z_dim, device=device)
    with torch.no_grad():
      image_frozen = generator_frozen(latent,
                                      c,
                                      truncation_psi=1.0,
                                      noise_mode='const')

    image = generator(latent,
                      c,
                      truncation_psi=1.0,
                      noise_mode='const')

    emb_delta_image = delta_image(models,
                                  image,
                                  image_frozen,
                                  device)

    emb_delta_text = delta_text(models,
                                text_target,
                                text_source,
                                device)
    # l2-регуляризация
    reg_loss = 0.0
    reg_count = 0
    for p_frozen, p in zip(generator_frozen.synthesis.parameters(),
                           generator.synthesis.parameters()):
      if p.requires_grad:
        reg_loss += (p_frozen.detach() - p).pow(2).sum()
        reg_count += p.numel()
    if reg_count > 0:
      reg_loss = reg_loss / (reg_count + 1e-8)
    else:
      reg_loss = torch.tensor(0.0, device=device)

    # Патч-регуляризация
    # Извлекаются патчи
    patches = extract_patches(image,
                              patch_size=64,
                              num_patches=8)
    patches_frozen = extract_patches(image_frozen,
                                     patch_size=64,
                                     num_patches=8)

    emb_patches = preprocessing_image(models,
                                      patches,
                                      device)
    emb_patches_frozen = preprocessing_image(models,
                                             patches_frozen,
                                             device)

    patch_loss = cosine_dist(emb_patches,
                             emb_patches_frozen)

    loss = criterion(emb_delta_image,
                     emb_delta_text)

    loss = loss + lambda_patch * patch_loss + lambda_l2 * reg_loss

    loss.backward()

    # Ограничивает норму градиентов
    torch.nn.utils.clip_grad_norm_(train_params, max_norm)

    optimizer.step()

    loss_lst.append(loss.item())

    print(f'Эпоха {epoch + 1}: loss={loss.item():.4f}')

    # Удаляет отработанные тензоры
    del latent, image_frozen, image, emb_delta_image, emb_delta_text
    del patches, patches_frozen, emb_patches_frozen, loss

    torch.cuda.empty_cache()

    if epoch % 50 == 0:
      image_sample = generator(latent_sample,
                               c,
                               truncation_psi=0.7,
                               noise_mode='const')
      img = (image_sample.clamp(-1, 1) + 1) / 2.0  # [-1, 1] → [0, 1]
      grid = make_grid(img, nrow=2)

      save_path = f'results/generated_image_epoch_{epoch}.jpg'
      save_image(grid, save_path, 'jpeg')

      plt.figure(figsize=(8, 8))
      plt.imshow(grid.permute(1, 2, 0).detach().cpu().numpy())
      plt.axis('off')
      plt.show()

      # Удаляет отработанные тензоры
      del image_sample, img

      torch.cuda.empty_cache()

  plt.figure(figsize=(8, 8))
  plt.plot(loss_lst)
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.title('График лосса')
  plt.grid(True)
  plt.show()

In [ ]:
train_generator(generator=G,
                generator_frozen=G_frozen,
                model_names=['ViT-B/32', 'ViT-B/16'],
                text_source='human',
                text_target='werewolf',
                epochs=301,
                criterion=clip_loss,
                device=device,
                lr=0.003,
                lr_w=0.002,
                lambda_patch=0.5,
                lambda_l2=0.1,
                max_norm=1.0,
                weights=[1.0, 1.0],
                use_conv_layers=True,
                c=None)